<a href="https://colab.research.google.com/github/zulhabibie/Data-Intelligent/blob/master/soal_1_test_bfi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
from datetime import datetime

# Sample data
data = {
    "id": ['OFA00000053', 'OFA00000054'],
    "customer": [
        "ABBAS, Mohammad Hasan (a.k.a. ABBAS, Mohammad Hassan; a.k.a. ABBAS, Muhammad), Damascus, Syria; DOB 01 Sep 1964; POB Al Ladhiqiyah, Syria; Gender Male (individual)",
        "ABDULAH, Hassanudin (a.k.a. ABDUL, Hasanudin; a.k.a. ABDULLAH, Hasanudin), Damascus, Syria; DOB 01 Sep 1964; POB Al Ladhiqiyah, Syria; Gender Male (individual)"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)


In [2]:
# Revised function to correctly extract Nama3 between "; a.k.a." and ")"
def extract_details_corrected(row):
    # Find all occurrences of names in parentheses and "a.k.a."
    aka_parts = re.findall(r'\(a\.k\.a\.\s([^)]+)', row['customer'])

    # Extract Nama1 before the first comma
    nama1 = row['customer'].split(' (a.k.a.')[0]

    # Initialize Nama2 and Nama3
    nama2 = nama3 = None
    # Assign Nama2 and Nama3 based on the number of "a.k.a." parts found
    if aka_parts:
        # Split the first "a.k.a." part to get Nama2 before the first semicolon
        nama2 = aka_parts[0].split(';')[0].strip()
        nama3 = aka_parts[0].split('; a.k.a. ')[1]

    # Extract Alamat, DOB, and POB using the same technique as previously defined
    alamat_search = re.search(r'\),\s(.+);\sDOB', row['customer'])
    alamat = alamat_search.group(1).strip() if alamat_search else ''

    dob_search = re.search(r'DOB\s(.+?);\sPOB', row['customer'])
    dob_text = dob_search.group(1).strip() if dob_search else ''
    dob = datetime.strptime(dob_text, '%d %b %Y').strftime('%Y%m%d') if dob_text else ''

    pob_search = re.search(r'POB\s(.+?);\sGender', row['customer'])
    pob = pob_search.group(1).strip() if pob_search else ''

    # Gender is always "Male" in the provided data, but let's extract it correctly
    gender_search = re.search(r'Gender\s(.+)\s\(individual\)', row['customer'])
    gender = gender_search.group(1).strip() if gender_search else ''

    return pd.Series([nama1, nama2, nama3, alamat, dob, pob, gender])

# Apply the corrected extraction function to the DataFrame
df[['Nama1', 'Nama2', 'Nama3', 'Alamat', 'DOB', 'POB', 'Gender']] = df.apply(extract_details_corrected, axis=1)

# Resulting DataFrame with the corrected columns
df_corrected = df[['id', 'Nama1', 'Nama2', 'Nama3', 'Alamat', 'DOB', 'POB', 'Gender']]
df_corrected


,id,Nama1,Nama2,Nama3,Alamat,DOB,POB,Gender
0,OFA00000053,"ABBAS, Mohammad Hasan","ABBAS, Mohammad Hassan","ABBAS, Muhammad","Damascus, Syria",19640901,"Al Ladhiqiyah, Syria",Male
1,OFA00000054,"ABDULAH, Hassanudin","ABDUL, Hasanudin","ABDULLAH, Hasanudin","Damascus, Syria",19640901,"Al Ladhiqiyah, Syria",Male
